In [6]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [7]:
%cd /storage/work/jjd330/local/bat_data/BatML/

/storage/work/j/jjd330/local/bat_data/BatML


In [8]:
from config import rt_dir
from ray_trace_funcs import RayTraces
from event2dpi_funcs import det2dpis, mask_detxy
# from do_InFoV_scan3 import Swift_Mask_Interactions, Source_Model_InFoV, Bkg_Model_wFlatA,\
#                             CompoundModel, Point_Source_Model_Binned_Rates,\
#                             theta_phi2imxy, bldmask2batxys, imxy2theta_phi,\
#                             get_fixture_struct
from flux_models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model
from models import Source_Model_InOutFoV, Bkg_Model_wFlatA,\
                CompoundModel, Point_Source_Model_Binned_Rates
from ray_trace_funcs import RayTraces
from sqlite_funcs import get_conn
from dbread_funcs import get_info_tab, get_twinds_tab
from do_manage import im_dist
from hp_funcs import ang_sep
from coord_conv_funcs import theta_phi2imxy, imxy2theta_phi, convert_imxy2radec,\
                            convert_radec2thetaphi, convert_radec2imxy
from do_llh_inFoV4realtime import parse_bkg_csv
from LLH import LLH_webins
from minimizers import NLLH_ScipyMinimize_Wjacob

In [4]:
%matplotlib inline

In [5]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]
ebins0 = np.round(ebins0, decimals=1)[:-1]
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print nebins

work_dir = '/storage/work/jjd330/local/bat_data/realtime_workdir/F646018360/'
conn = get_conn(os.path.join(work_dir,'results.db'))
info_tab = get_info_tab(conn)
trigger_time = info_tab['trigtimeMET'][0]
print trigger_time

evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print att_quat

ndets = np.sum(dmask==0)
print "Ndets: ", np.sum(dmask==0)

t_end = trigger_time + 1e3
t_start = trigger_time - 1e3
mask_vals = mask_detxy(dmask, ev_data)
bl_dmask = (dmask==0.)

bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

print np.sum(bl_ev)
ev_data0 = ev_data[bl_ev]

9
646018383.1787
[-0.03597053  0.2345147  -0.64420835  0.72712074]
Ndets:  14932
1367885


In [9]:
ra, dec = 233.117, -26.213 
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)
imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

(38.54132137017975, 137.65241966813443)
(-0.5887551341212707, -0.5366203642198198)


In [10]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

In [11]:
rt_obj = RayTraces(rt_dir)

INFO:root:0.02
INFO:root:0.002


In [27]:
rt = rt_obj.get_intp_rt(imx, imy)
print np.shape(rt)

(173, 286)


In [12]:
%%time
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)

CPU times: user 726 µs, sys: 0 ns, total: 726 µs
Wall time: 601 µs


In [13]:
%%time
sig_mod.set_theta_phi(theta, phi)

INFO:root:Making new response object


(0.0, 30.48, -14.117)
(0.0, 30.48, -14.117)
36.0 45.0
2.652419668134428
42.34758033186557
DEBUG:root:opening file: fwd_ray_trace_-0.60000_-0.54000_-0.58000_-0.52000_.npy


response.py:1499: RuntimeWarning: invalid value encountered in divide
  (self.orig_photonEs[self.Einds1] - self.orig_photonEs[self.Einds0])


DEBUG:root:nbytes_total=95789408.0
max rt: 0.8604
DEBUG:root:Openning file: resp_by_sand_hpind_46.fits
DEBUG:root:Openning file: resp_by_sand_hpind_47.fits
DEBUG:root:Openning file: resp_by_sand_theta_0.fits
initing ResponseDPI, with fname
/storage/work/jjd330/local/bat_data/resp_tabs_ebins/drm_theta_36.0_phi_30.0_.fits
initing ResponseDPI, with fname
/storage/work/jjd330/local/bat_data/resp_tabs_ebins/drm_theta_36.0_phi_45.0_.fits
initing ResponseDPI, with fname
/storage/work/jjd330/local/bat_data/resp_tabs_ebins/drm_theta_45.0_phi_30.0_.fits
initing ResponseDPI, with fname
/storage/work/jjd330/local/bat_data/resp_tabs_ebins/drm_theta_45.0_phi_45.0_.fits
CPU times: user 4.07 s, sys: 2.01 s, total: 6.08 s
Wall time: 10.2 s


In [16]:
from config import solid_angle_dpi_fname
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']
INFO:root:0.02
INFO:root:0.002
DEBUG:root:opening file: fwd_ray_trace_-0.10000_-0.48000_-0.08000_-0.46000_.npy
DEBUG:root:nbytes_total=95789408.0
DEBUG:root:opening file: fwd_ray_trace_-0.12000_-0.54000_-0.10000_-0.52000_.npy
DEBUG:root:nbytes_total=191578816.0
DEBUG:root:opening file: fwd_ray_trace_0.00000_-0.70000_0.02000_-0.68000_.npy
DEBUG:root:nbytes_total=287368224.0
DEBUG:root:opening file: fwd_ray_trace_-0.50000_-0.22000_-0.48000_-0.20000_.npy
DEBUG:root:nbytes_total=383157632.0


{'4U 1700-377_imx': -0.09814853057709713,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.012490135921897544,
 '4U 1700-377_rate_1': 0.00724252648862388,
 '4U 1700-377_rate_2': 0.004529564586462892,
 '4U 1700-377_rate_3': 0.0016876913514016756,
 '4U 1700-377_rate_4': 0.0013255091059552946,
 '4U 1700-377_rate_5': 0.0012956105576278066,
 '4U 1700-377_rate_6': 0.0005834459169539205,
 '4U 1700-377_rate_7': 5e-08,
 '4U 1700-377_rate_8': 0.00014013200316638987,
 'Background_bkg_rate_0': 0.09137032207011836,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.040089856902610514,
 'Background_bkg_rate_3': 0.039491993420780136,
 'Background_bkg_rate_4': 0.03467802668352813,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.03360249200395507,
 'Background_bkg_rate_7': 0.024861082684787414,
 'Background_bkg_rate_8': 0.01763994761450761,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.18596326494289767,
 

In [17]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in bkg_params.iteritems():
    pars_[bkg_name+'_'+pname] = val
for pname,val in flux_params.iteritems():
    pars_['Signal_'+pname] = val
pars_

{'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_imx': -0.09814853057709713,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_imy': -0.4742076074486664,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_0': 0.012490135921897544,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_1': 0.00724252648862388,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_2': 0.004529564586462892,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_3': 0.0016876913514016756,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_4': 0.0013255091059552946,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_5': 0.0012956105576278066,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_6': 0.0005834459169539205,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_4U 1700-377_rate_7': 5e-08,
 'Background+4U 1700-377+GRO J1655-40+G

In [18]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

sig_miner = NLLH_ScipyMinimize_Wjacob('')

sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)

sig_llh_obj.set_model(comp_mod)

sig_miner.set_llh(sig_llh_obj)

In [19]:
fixed_pnames = pars_.keys()
fixed_vals = pars_.values()
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

In [20]:
%%time
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

CPU times: user 92.4 ms, sys: 994 µs, total: 93.4 ms
Wall time: 127 ms


In [21]:
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

In [22]:
%%time
pars, nllh, res = sig_miner.minimize()

CPU times: user 93.4 ms, sys: 2.39 ms, total: 95.8 ms
Wall time: 113 ms


In [24]:
print(res)
print(nllh)
print(pars)

[      fun: 45894.07068100057
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00239975])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
   status: 0
  success: True
        x: array([0.006875])]
[45894.07068100057]
[[0.006875003655224621]]


In [26]:
%%time
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print bkg_nllh
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print sqrtTS

46038.7433272
17.010152627576055
CPU times: user 9.21 ms, sys: 0 ns, total: 9.21 ms
Wall time: 8.7 ms
